# Generate long signal and share it

## Define the environment

* Firstly, let's add WaveBeans dependencies as well as logging library.
* Secondly, add all imports that might be useful.
* Lastly, initialize parameters

In [1]:
//uncomment if you're trying to access the local maven repository and have it mounted.
//@file:Repository("file:///home/jovyan/maven-local/repository")
@file:DependsOn("io.wavebeans:lib:0.3.0")
@file:DependsOn("io.wavebeans:exe:0.3.0")
@file:DependsOn("io.wavebeans:http:0.3.0")
// uncomment if you need logging
// @file:DependsOn("ch.qos.logback:logback-classic:1.2.3")
// @file:DependsOn("ch.qos.logback:logback-core:1.2.3")

import io.wavebeans.lib.*
import io.wavebeans.lib.io.*
import io.wavebeans.lib.math.*
import io.wavebeans.lib.stream.*
import io.wavebeans.lib.stream.fft.*
import io.wavebeans.lib.stream.window.*
import io.wavebeans.lib.table.*
import io.wavebeans.http.*
import io.wavebeans.execution.*
import java.util.concurrent.TimeUnit.*

val windowSize = 801
val stepSize = 256
val fftSize = 1024

## Prepare for sharing

* Start the HTTP Service on port 6800, so we could query all data in a different Jupyter notebook

In [2]:
HttpService(serverPort = 6800).start()

io.wavebeans.http.HttpService@23e6cf6f

## Define the stream and execute it

* Define our signal processing stream:
    * it is infinite;
    * we store only last 2 minutes in the table.

In [3]:
val signal = (880.sine() + 440.sine() + 220.sine())
    .window(windowSize, step = stepSize)

val triangularFft = signal
    .triangular()
    .fft(fftSize)
    .toTable("fft-triangular", 2.m)
val hammingFft = signal
    .hamming()
    .fft(fftSize)
    .toTable("fft-hamming", 2.m)

* Execute the stream, it'll never finish though

In [ ]:
val overseer = LocalDistributedOverseer(
    listOf(triangularFft, hammingFft), 
    threadsCount = 2,
    partitionsCount = 2
)

val errors = overseer.eval(44100.0f)
    .map { it.get().exception }
    .filterNotNull()
    .toList()
    
overseer.close()

if (errors.isEmpty()) {
    println("SUCCESS")
} else {
    println("ERROR:\n" + errors.joinToString("\n"))
}
